# Create data for stacking code testing

In [ ]:
import numpy as np
from pathlib import Path
from astropy.io import fits
from astropy.table import Table, vstack, hstack, unique,join
import matplotlib.pyplot as plt
import pandas as pd
import sys

# sys.path.append("/global/u2/b/bid13/VI/prospect/py")

import desispec.coaddition
import desispec.io
import desispec.spectra
# from desitarget.cmx.cmx_targetmask import cmx_mask
from desitarget.sv1.sv1_targetmask import desi_mask
from prospect import utilities ,plotframes
# from prospect_my import plotframes

In [ ]:
data_path = Path("/global/cfs/cdirs/desi/spectro/redux/cascades/tiles/")
my_path = Path("/global/cscratch1/sd/bid13/stack-spectra")

In [ ]:
tile = "80605"
date = "20201215"
tile_path = data_path / tile /"deep"

z_table = Table()
for file_path in tile_path.glob("zbest-*"):
    z_petal = Table.read(file_path, hdu="ZBEST")
    fibermap_petal = Table.read(file_path, hdu="FIBERMAP")
    mask = ((fibermap_petal["SV1_DESI_TARGET"] & desi_mask.mask("LRG"))>0) & (fibermap_petal["FIBERSTATUS"] == 0)
    fibermap_petal = fibermap_petal[mask]
    fibermap_petal = unique(fibermap_petal, keys="TARGETID")
    
    spec = desispec.io.read_spectra(str(file_path).replace("zbest","coadd") )
    spec = spec.select(targets= fibermap_petal["TARGETID"])
    snr_b = np.median( spec.flux["b"]*np.sqrt(spec.ivar["b"]), axis=-1)
    fibermap_petal["SNR_B"] = snr_b
    snr_r = np.median( spec.flux["r"]*np.sqrt(spec.ivar["r"]), axis=-1)
    fibermap_petal["SNR_R"] = snr_r
    snr_z = np.median( spec.flux["z"]*np.sqrt(spec.ivar["z"]), axis=-1)
    fibermap_petal["SNR_Z"] = snr_z
    fibermap_petal["SNR_MAX"] = np.max([snr_b,snr_r,snr_z], axis=0)
    merged_table = join(z_petal, fibermap_petal, keys="TARGETID", metadata_conflicts="silent")
    
    z_table = vstack([z_table, merged_table])

In [ ]:
_=plt.hist(z_table["Z"], histtype="step", color="k", bins=50)
plt.xlabel("Spec-Z")

In [ ]:
zmag = 22.5 - 2.5*np.log10(z_table["FIBERTOTFLUX_Z"]) #/z_table["MW_TRANSMISSION_Z"])
plt.figure(figsize=(15,10))
plt.scatter(zmag, z_table["SNR_MAX"], marker=".", s=0.8, color="k")
# plt.axhline(2, c="k", ls ="--")
plt.xlim(16, 23)
plt.ylim(-1,50)
plt.xlabel("z Fiber total mag", size=15)
plt.ylabel("SNR_MAX", size=15)


In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(zmag, z_table["DELTACHI2"], marker=".", s=0.8, color="k")
plt.yscale("log")
plt.axhline(9, c="k", ls ="--")
plt.xlim(16, 23)
# plt.ylim(-1,50)
plt.xlabel("z Fiber total mag", size=15)
plt.ylabel("Delta Chi ^2", size=15)

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(z_table["Z"], z_table["DELTACHI2"], marker=".", s=0.8, color="k")
plt.yscale("log")
plt.axhline(9, c="k", ls ="--")
plt.xlim(-0.1, 2)
plt.xlabel("Spec-Z")
plt.ylabel("Delta Chi ^2", size=15)

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(z_table["SNR_MAX"], z_table["DELTACHI2"], marker=".", s=0.8, color="k")
plt.yscale("log")
# plt.axhline(15, c="k", ls ="--")
plt.xlim(-1,50)
plt.xlabel("SNR_MAX")
plt.ylabel("Delta Chi ^2", size=15)

In [ ]:
# Select objects with redshift 1-ish
z_targets = z_table[(z_table["Z"]>=1)&(z_table["Z"]<=1.2)&(z_table["DELTACHI2"]>15)]["TARGETID"]

In [ ]:
obs_db = utilities.make_targetdict(str(data_path), tiles=[tile], nights=[date]) # petals, tiles = optional arguments

spectra, zcat= utilities.load_spectra_zcat_from_targets(z_targets, str(data_path), obs_db)

In [ ]:
# zcat.write(my_path/"zcat.fits")
# desispec.io.write_spectra(my_path/"spectra.fits", spectra)